<a href="https://colab.research.google.com/github/MaxB32/multi-purpose-neural-network/blob/main/Neural_network_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load in necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#1. Neccessary functions for parts of a neural network

## 1.1 - Error functions

### 1.1.1 - Regression cost function (mean squared error)

In [ ]:
# Define MSE function
def mean_squared_error(y_true, y_hat):
  # Define length of data
  m = len(y_true)
  # Calculate error for individual parameters
  loss_function = (y_true - y_hat)**2
  # Calculate average to acquire cost
  cost_function = np.mean(loss_function)
  return cost_function

### 1.1.2 - Binary classification cost function (Binary cross entropy)

In [ ]:
def binary_cross_entropy(y_true, y_hat):
  # Extract length of input
  m = len(y_true)
  # Caluclate loss function (binary cross entropy error per instance)
  loss_function = (y_true * np.log(y_hat) + (1 - y_true) *np.log(1 - y_hat))
  # Take mean to acquire cost of all instances
  cost_function = - np.mean(loss_function)
  return cost_function

### 1.1.3 - Multi-class classification cost function (Classification cross entropy)

In [ ]:
def softmax_pro(logits):
  # Calculate softmax probabilities
  expo_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
  softmax_prob = expo_logits / np.sum(expo_logits, axis=1, keepdims=True)
  return softmax_prob


def multi_cross_entropy(y_true, y_hat):

  # Clip predictions to ensure no log(0)
  y_hat = np.clip(y_hat, 1e-12, 1)

  loss_function = -np.sum(y_true * np.log(y_hat), axis = 1)

  cost_function = np.mean(loss_function)

  return cost_function

## 1.2 - Derivatives of cost functions

### 1.2.1 - Derivative of mean squared error

In [ ]:
def derivative_MSE(y_hat, y_true):
  m = y_true.shape[0]
  delta_MSE = (2/m) * (y_hat - y_true)
  return delta_MSE

### 1.2.2 - Derivative of binary cross entropy

In [ ]:
def derivative_binary_XEntropy(y_hat, y_true):
  delta_bin_XEntropy = (- y_true / y_hat) + ((1 - y_true) / (1 - y_hat))
  return np.mean(delta_bin_XEntropy)

### 1.2.3 - Derivative of classification cross entropy

In [ ]:
def derivative_class_XEntropy(y_true, y_hat):
  delta_class_XEntropy = -(y_true / y_hat)
  return np.mean(delta_class_XEntropy)

## 1.3 - Activation functions

### 1.3.1 - Sigmoid activation function

In [ ]:
def sigmoid_func(z):
  sigma_output = 1 / (1 + np.exp(-z))
  return sigma_output

### 1.3.2 - ReLu activation function

In [ ]:
def Relu_func_manual(z):
  Relu_output = 0
  if z > 0:
    Relu_output = z
  else:
    Relu_output = 0
  return Relu_output

In [ ]:
def Relu_func_auto(z):
  Relu_output = np.maximum(0, z)
  return Relu_output

## 1.4 - Derivatives of activation functions

### 1.4.1 - Derivative of sigmoid function

In [ ]:
def derivative_sigmoid(sigma_output):
  delta_sigma = sigma_output(1 - sigma_output)
  return delta_sigma

### 1.5.2 - Derivative of Relu activation function

In [ ]:
def derivative_Relu(Relu_output):
  if Relu_output > 1:
    delta_Relu = 1
  else:
    delta_Relu = 0
  return delta_Relu

## 1.5 - Training neural network

### 1.5.1 - Forward pass

In [ ]:
# Define forward pass training function
def forward_pass(n_neurons, n_features, n_instances, n_layers, X, activation_func):
  # Define bias
  b = np.ones((n_instances, 1))
  # Append bias to input matrix
  X_b = np.hstack([X, b])
  activations = []

  current_activations = X_b
  activations.append(current_activation)
  for L in range(n_layers):
    weight_matrix = weight_matrices[L]
    # Calculate activation
    z = current_activation@weight_matrix
    # Apply activation function to activation
    if activation_func == 'sigmoid':
      current_activation = sigmoid_func(z)
    elif activation_func == 'Relu':
      current_activation = Relu_func_auto(z)
    else:
      raise ValueError('Unknown activation function')
    activations.append(current_activation)
  return activations, weight_matrices

### 1.5.2 - Backpropagation

In [ ]:
def backpropagation(logits, alpha, y_true, y_hat, activation_func, error_func, X, n_layers, activations, weight_matrices):
  m = len(y_true)
  # Compute gradient of loss with repsect to output (error)
  if error_func == 'MSE':
    delta_loss_activations = derivative_MSE(y_hat, y_true)
  elif error_func == 'binary_Xentropy':
    delta_loss_activations = derivative_binary_XEntropy(y_hat, y_true)
  else:
    delta_loss_activations = derivative_class_XEntropy(y_hat, y_true)
  # Compute gradient of activations with respect to weight sums (z)
  if activation_func == 'sigmoid':
    delta_activations_WeightdSums = derivative_sigmoid(y_hat)
  else:
    delta_activations_WeightdSums = derivative_Relu(y_hat)
  # Compute gradient of loss with respect to the weighted sums (z)
  delta_current_layer = np.dot(delta_loss_activations, delta_activations_WeightdSums)
  # Compute gradient of loss with respect to weights
  delta_loss_weights = np.dot(delta_current_layer.T, X) / m
  for L in reversed(range(1, n_layers)):
    previous_activation = activations[L - 1]
    if activation_func == 'sigmoid':
      delta_previous_layer = np.dot(delta_current_layer, weight_matrix[L].T) * derivative_sigmoid(previous_activation)
    else:
      delta_previous_layer = np.dot(delta_current_layer, weight_matrix[L].T) * derivative_Relu(previous_activation)
    weight_matrices[L] -= alpha * np.dot(previous_activation.T, delta_previous_layer)

    delta_current_layer = delta_previous_layer
    previous_activation = activations[L - 1]
    return weight_matrices

# 1.6 - Building neural network class

In [ ]:
# Define the class for a multi-purpose neural network
class multi_purpose_NN:
    # Initialize the class with hyperparameters and network structure
    def __init__(self, alpha, activation_func, error_func, n_neurons, n_features, n_instances, n_layers):
        # Assign input arguments to instance variables
        self.alpha = alpha
        self.activation_func = activation_func
        self.error_func = error_func
        self.n_neurons = n_neurons
        self.n_features = n_features
        self.n_instances = n_instances
        self.n_layers = n_layers

        # Initialize weights dynamically using Xavier initialization
        self.weights_matrices = []
        input_dim = n_features + 1  # Add 1 to include the bias term
        for layer in range(n_layers):
            # Determine the output dimension based on layer type
            if layer == n_layers - 1 and self.error_func == "MSE":
                output_dim = 1  # Single output for regression
            elif layer == n_layers - 1 and self.error_func == "class_Xentropy":
                output_dim = 3  # Number of classes for classification
            else:
                output_dim = n_neurons

            # Create a weight matrix with random initialization
            weight_matrix = np.random.randn(input_dim, output_dim) * np.sqrt(1 / input_dim)
            self.weights_matrices.append(weight_matrix)

            # Update the input dimension for the next layer
            input_dim = output_dim

    # Perform a forward pass through the network
    def forward_pass(self, X):
        # Add a bias term to the input
        b = np.ones((X.shape[0], 1))
        X_b = np.hstack([X, b])
        activations = []
        current_activation = X_b
        activations.append(current_activation)

        # Compute activations for each layer
        for L in range(self.n_layers):
            weight_matrix = self.weights_matrices[L]
            z = current_activation @ weight_matrix

            # Apply the appropriate activation function
            if L == self.n_layers - 1 and self.error_func == 'class_Xentropy':
                current_activation = self.softmax_func(z)
            elif self.activation_func == 'sigmoid':
                current_activation = self.sigmoid_func(z)
            elif self.activation_func == 'Relu':
                current_activation = self.Relu_func_auto(z)
            else:
                raise ValueError('Unknown activation function')

            activations.append(current_activation)
        return activations

    # Perform backpropagation to update weights
    def backpropagation(self, X, y_true, y_hat, activations):
        # Calculate the number of training examples
        m = y_true.shape[0]

        # Compute the gradient for the output layer based on the loss function
        if self.error_func == 'MSE':
            delta_loss_activations = self.derivative_MSE(y_hat, y_true)
            if self.activation_func == 'sigmoid':
                delta_activations_WeightdSums = self.derivative_sigmoid(y_hat)
            elif self.activation_func == 'Relu':
                delta_activations_WeightdSums = np.where(y_hat > 0, 1, 0)
            else:
                raise ValueError('Unsupported activation function for MSE')
            delta_current_layer = delta_loss_activations * delta_activations_WeightdSums
        elif self.error_func == 'class_Xentropy':
            delta_current_layer = y_hat - y_true
        elif self.error_func == 'binary_Xentropy':
            delta_loss_activations = self.derivative_binary_XEntropy(y_hat, y_true)
            if self.activation_func == 'sigmoid':
                delta_current_layer = delta_loss_activations * self.derivative_sigmoid(y_hat)
            else:
                raise ValueError('Unsupported activation function for binary cross-entropy')
        else:
            raise ValueError('Unsupported error function')

        # Update the weights for the output layer
        gradient_output_layer = np.dot(activations[-2].T, delta_current_layer) / m
        self.weights_matrices[-1] -= self.alpha * gradient_output_layer

        # Backpropagate through the hidden layers
        for L in reversed(range(self.n_layers - 1)):
            delta_previous_layer = np.dot(delta_current_layer, self.weights_matrices[L + 1].T)
            if self.activation_func == 'sigmoid':
                delta_previous_layer *= self.derivative_sigmoid(activations[L + 1])
            elif self.activation_func == 'Relu':
                delta_previous_layer *= np.where(activations[L + 1] > 0, 1, 0)
            else:
                raise ValueError('Unsupported activation function for hidden layers')

            gradient_hidden_layer = np.dot(activations[L].T, delta_previous_layer) / m
            self.weights_matrices[L] -= self.alpha * gradient_hidden_layer
            delta_current_layer = delta_previous_layer

    # Train the neural network
    def train(self, X, y, epochs):
        # Ensure correct format for the labels
        if self.error_func == "class_Xentropy":
            if len(y.shape) != 2 or y.shape[1] != self.weights_matrices[-1].shape[1]:
                raise ValueError(f"Expected one-hot encoded labels with {self.weights_matrices[-1].shape[1]} classes, but got shape {y.shape}")
        elif self.error_func == "MSE":
            y = y.reshape(-1, 1)

        for epoch in range(epochs):
            # Perform forward pass
            activations = self.forward_pass(X)
            y_hat = activations[-1]

            # Check alignment of predicted and true labels
            if y.shape != y_hat.shape:
                raise ValueError(f"Shape mismatch: y_true shape = {y.shape}, y_hat shape = {y_hat.shape}")

            # Perform backpropagation
            self.backpropagation(X, y, y_hat, activations)

            # Print loss every 100 epochs
            if epoch % 100 == 0:
                loss = self.compute_loss(y_hat, y)
                print(f"Epoch {epoch}, Loss: {loss}")

    # Make predictions using the trained model
    def predict(self, X):
        activations = self.forward_pass(X)
        return activations[-1]

    # Compute the loss based on the error function
    def compute_loss(self, y_hat, y_true):
        if self.error_func == 'MSE':
            loss = np.mean((y_hat - y_true) ** 2)
        elif self.error_func == 'class_Xentropy':
            y_hat = np.clip(y_hat, 1e-12, 1 - 1e-12)
            loss = -np.mean(np.sum(y_true * np.log(y_hat), axis=1))
        elif self.error_func == 'binary_Xentropy':
            y_hat = np.clip(y_hat, 1e-12, 1 - 1e-12)
            loss = -np.mean(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat))
        else:
            raise ValueError('Unknown error function type')
        return loss

    # Define activation and helper functions
    def sigmoid_func(self, z):
        return 1 / (1 + np.exp(-z))

    def Relu_func_auto(self, z):
        return np.maximum(0, z)

    def softmax_func(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def derivative_MSE(self, y_hat, y_true):
        return 2 * (y_hat - y_true) / y_true.shape[0]

    def derivative_sigmoid(self, sigma_output):
        return sigma_output * (1 - sigma_output)

    # Calculate accuracy for classification tasks
    def calculate_accuracy(self, y_true, y_pred):
        correct_predictions = np.sum(np.argmax(y_pred, axis=1) == np.argmax(y_true, axis=1))
        total_predictions = y_true.shape[0]
        return correct_predictions / total_predictions

    # Calculate precision for classification tasks
    def calculate_precision(self, y_true, y_pred):
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_true, axis=1)
        precision_scores = []
        for class_label in np.unique(y_true_classes):
            true_positives = np.sum((y_pred_classes == class_label) & (y_true_classes == class_label))
            predicted_positives = np.sum(y_pred_classes == class_label)
            precision = true_positives / predicted_positives if predicted_positives > 0 else 0
            precision_scores.append(precision)
        return precision_scores

    # Calculate Root Mean Square Error
    def calculate_rmse(self, y_true, y_pred):
        return np.sqrt(np.mean((y_true - y_pred) ** 2))

#2 - Loading in data for testing

## 2.1 - Classification data

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split into train and test sets
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y, test_size=0.2, random_state=42)

## 2.2 - Regression data

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
housing = fetch_california_housing()
X = housing.data
y = housing.target

# Split into train and test sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)


#3 - Neural network initial training and testing

## 3.1 - Basic classification neural network (3 neurons and 2 layers)

In [ ]:
# Initialize and train the model
nn_classification = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=3, n_features=4, n_instances=120, n_layers=2)

nn_classification.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification.predict(X_test_class)
accuracy = nn_classification.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 1.5653303104659433
Epoch 100, Loss: 0.8809943450023492
Epoch 200, Loss: 0.8228411751004551
Epoch 300, Loss: 0.79423159033933
Epoch 400, Loss: 0.7809486708292527
Epoch 500, Loss: 0.7717379580150736
Epoch 600, Loss: 0.7645456047606151
Epoch 700, Loss: 0.7588677373151416
Epoch 800, Loss: 0.7544542445798861
Epoch 900, Loss: 0.7509829080820993
Classification Accuracy: 0.3333333333333333
Classification Precision (Per Class): [0.3333333333333333, 0, 0]


## 3.2 - Basic regression neural network (3 neurons and 2 layers)

In [ ]:
# Before initializing the neural network
print(f"X_train_reg shape: {X_train_reg.shape}")  # Print the shape of the training data
print(f"Number of features in X_train_reg: {X_train_reg.shape[1]}")  # Print the number of features

nn_regression = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=3,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=2
)


# After initialization, print the network configuration
print(f"n_features in neural network: {nn_regression.n_features}")
print(f"Expected input size in neural network: {nn_regression.n_features + 1} (including bias term)")
print(f"Weight matrix for Layer 0: {nn_regression.weights_matrices[0].shape}")

X_train_reg shape: (16512, 8)
Number of features in X_train_reg: 8
n_features in neural network: 8
Expected input size in neural network: 9 (including bias term)
Weight matrix for Layer 0: (9, 3)


In [ ]:
# Train the regression model
epochs = 1000
nn_regression.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 3.5685820538712556
Epoch 100, Loss: 3.568557899856682
Epoch 200, Loss: 3.56853374643444
Epoch 300, Loss: 3.568509593604529
Epoch 400, Loss: 3.568485441366948
Epoch 500, Loss: 3.5684612897216965
Epoch 600, Loss: 3.568437138668775
Epoch 700, Loss: 3.5684129882081823
Epoch 800, Loss: 3.5683888383399185
Epoch 900, Loss: 3.568364689063982
Root Mean Square Error (RMSE): 1.8693


# 4 - Neuron experimentation

Here the number of layers will be held constant at 2, all experiments won't be optimal due to timing scenarios and reducing code to impleemnt a random search or similar optimality algorithm

## 4.1 - Classification 12 neurons

In [ ]:
# Initialize and train the model
nn_classification_12 = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=12, n_features=4, n_instances=120, n_layers=2)

nn_classification_12.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_12.predict(X_test_class)
accuracy = nn_classification_12.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_12.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 3.017796718416685
Epoch 100, Loss: 0.6014516353211348
Epoch 200, Loss: 0.4619003366914164
Epoch 300, Loss: 0.3968984750901633
Epoch 400, Loss: 0.351598377383943
Epoch 500, Loss: 0.31332243031273693
Epoch 600, Loss: 0.27952591827361256
Epoch 700, Loss: 0.2501118469752322
Epoch 800, Loss: 0.22487008110492285
Epoch 900, Loss: 0.2036461180674146
Classification Accuracy: 1.0
Classification Precision (Per Class): [1.0, 1.0, 1.0]


## 4.2 - Classiciation 24 neurons

In [ ]:
# Initialize and train the model
nn_classification_24 = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=24, n_features=4, n_instances=120, n_layers=2)

nn_classification_24.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_24.predict(X_test_class)
accuracy = nn_classification_24.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_24.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 2.2931964092354624
Epoch 100, Loss: 0.4848314681191515
Epoch 200, Loss: 0.37761436756651345
Epoch 300, Loss: 0.31088110277585995
Epoch 400, Loss: 0.26341045513102723
Epoch 500, Loss: 0.22843406177669465
Epoch 600, Loss: 0.20210841633091883
Epoch 700, Loss: 0.1819090348521541
Epoch 800, Loss: 0.16611343657162753
Epoch 900, Loss: 0.15353808297188984
Classification Accuracy: 0.9333333333333333
Classification Precision (Per Class): [1.0, 0.8181818181818182, 1.0]


## 4.3 - Classification 36 neurons

In [ ]:
# Initialize and train the model
nn_classification_36 = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=36, n_features=4, n_instances=120, n_layers=2)

nn_classification_36.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_36.predict(X_test_class)
accuracy = nn_classification_36.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_36.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 1.495927703653554
Epoch 100, Loss: 0.38598476829362494
Epoch 200, Loss: 0.30468565335109654
Epoch 300, Loss: 0.2546955730392503
Epoch 400, Loss: 0.21858716966245478
Epoch 500, Loss: 0.19179413422632013
Epoch 600, Loss: 0.17165003704837078
Epoch 700, Loss: 0.15629618324307384
Epoch 800, Loss: 0.14421738262994588
Epoch 900, Loss: 0.13452069516147794
Classification Accuracy: 1.0
Classification Precision (Per Class): [1.0, 1.0, 1.0]


## 4.4 - Regression 12 neurons

In [ ]:
nn_regression_12 = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=12,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=2
)

# Train the regression model
epochs = 1000
nn_regression_12.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_12.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_12.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 4.223217447647459
Epoch 100, Loss: 4.2231032363011645
Epoch 200, Loss: 4.2229890209747865
Epoch 300, Loss: 4.222874801670544
Epoch 400, Loss: 4.22276057839067
Epoch 500, Loss: 4.222646351137383
Epoch 600, Loss: 4.222532119912916
Epoch 700, Loss: 4.222417884719492
Epoch 800, Loss: 4.22230364555934
Epoch 900, Loss: 4.22218940243469
Root Mean Square Error (RMSE): 2.0324


## 4.5 - Regression 24 neurons

In [ ]:
nn_regression_24 = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=24,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=2
)

# Train the regression model
epochs = 1000
nn_regression_24.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_24.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_24.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 4.075664801932745
Epoch 100, Loss: 4.075428059074445
Epoch 200, Loss: 4.075191314185063
Epoch 300, Loss: 4.074954567286191
Epoch 400, Loss: 4.074717818399423
Epoch 500, Loss: 4.07448106754635
Epoch 600, Loss: 4.074244314748569
Epoch 700, Loss: 4.074007560027675
Epoch 800, Loss: 4.073770803405266
Epoch 900, Loss: 4.0735340449029405
Root Mean Square Error (RMSE): 1.9891


## 4.6 - Regression 36 neurons

In [ ]:
nn_regression_36 = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=36,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=2
)

# Train the regression model
epochs = 1000
nn_regression_36.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_36.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_36.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 3.9603495467145997
Epoch 100, Loss: 3.960018876702875
Epoch 200, Loss: 3.959688223536534
Epoch 300, Loss: 3.9593575872738995
Epoch 400, Loss: 3.9590269679732777
Epoch 500, Loss: 3.9586963656929575
Epoch 600, Loss: 3.9583657804912113
Epoch 700, Loss: 3.958035212426296
Epoch 800, Loss: 3.957704661556452
Epoch 900, Loss: 3.9573741279399015
Root Mean Square Error (RMSE): 1.9652


# 5 - Layers experimentation

Similar to the neuron experiment the purpose of this experiment is more to see if the nuerl network algorithm can indeed update dynamically the number of layers to create different neural network architectures rather than fidning optimal solutions for the datasets.

## 5.1 - Classification 3 layer experiment, where we will use max number of neurons experimneted with at a number of 36

In [ ]:
# Initialize and train the model
nn_classification_3Layer = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=36, n_features=4, n_instances=120, n_layers=3)

nn_classification_3Layer.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_3Layer.predict(X_test_class)
accuracy = nn_classification_3Layer.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_3Layer.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 1.3424253815457083
Epoch 100, Loss: 0.3756079093160711
Epoch 200, Loss: 0.2505125214884822
Epoch 300, Loss: 0.1831070986719854
Epoch 400, Loss: 0.1440885996149338
Epoch 500, Loss: 0.12063165998329088
Epoch 600, Loss: 0.10569302065102656
Epoch 700, Loss: 0.09564526078486689
Epoch 800, Loss: 0.0885714126997005
Epoch 900, Loss: 0.08340495421385977
Classification Accuracy: 0.9333333333333333
Classification Precision (Per Class): [1.0, 0.8888888888888888, 0.9090909090909091]


## 5.2 - Classification 4 layers

In [ ]:
# Initialize and train the model
nn_classification_4Layer = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=36, n_features=4, n_instances=120, n_layers=4)

nn_classification_4Layer.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_4Layer.predict(X_test_class)
accuracy = nn_classification_4Layer.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_4Layer.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 1.2092956482604724
Epoch 100, Loss: 0.43444877206992744
Epoch 200, Loss: 0.2727077098497286
Epoch 300, Loss: 0.1729378482208787
Epoch 400, Loss: 0.12798445878362424
Epoch 500, Loss: 0.10637806059948171
Epoch 600, Loss: 0.09468986081502301
Epoch 700, Loss: 0.08763785420755796
Epoch 800, Loss: 0.08298288971957914
Epoch 900, Loss: 0.07968016975219049
Classification Accuracy: 0.9333333333333333
Classification Precision (Per Class): [1.0, 0.8181818181818182, 1.0]


## 5.3 - Classification 5 layers

In [ ]:
# Initialize and train the model
nn_classification_5Layer = multi_purpose_NN(alpha=0.01, activation_func='Relu', error_func='class_Xentropy',
                                     n_neurons=36, n_features=4, n_instances=120, n_layers=5)

nn_classification_5Layer.train(X_train_class, y_train_class, epochs=1000)

# Predict and evaluate
y_pred_test_class = nn_classification_5Layer.predict(X_test_class)
accuracy = nn_classification_5Layer.calculate_accuracy(y_test_class, y_pred_test_class)
precision = nn_classification_5Layer.calculate_precision(y_test_class, y_pred_test_class)

print(f"Classification Accuracy: {accuracy}")
print(f"Classification Precision (Per Class): {precision}")

Epoch 0, Loss: 1.1803129538229755
Epoch 100, Loss: 0.44900718871160916
Epoch 200, Loss: 0.20949110081944558
Epoch 300, Loss: 0.12738373601906858
Epoch 400, Loss: 0.10020873425958277
Epoch 500, Loss: 1.0870197238116712
Epoch 600, Loss: 0.1909030521339265
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan
Classification Accuracy: 0.3333333333333333
Classification Precision (Per Class): [0.3333333333333333, 0, 0]


## 5.4 - Regression 3 layers

In [ ]:
nn_regression_3Layer = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=36,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=3
)

# Train the regression model
epochs = 1000
nn_regression_3Layer.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_3Layer.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_3Layer.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 3.703758850431712
Epoch 100, Loss: 3.703410713144286
Epoch 200, Loss: 3.7030626479963162
Epoch 300, Loss: 3.7027146550377545
Epoch 400, Loss: 3.702366734318479
Epoch 500, Loss: 3.7020188858882963
Epoch 600, Loss: 3.7016711097969393
Epoch 700, Loss: 3.701323406094069
Epoch 800, Loss: 3.700975774829271
Epoch 900, Loss: 3.7006282160520634
Root Mean Square Error (RMSE): 1.9040


## 5.5 - Regression 4 layers

In [ ]:
nn_regression_4Layer = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=36,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=4
)

# Train the regression model
epochs = 1000
nn_regression_4Layer.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_4Layer.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_4Layer.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 3.9695291215442774
Epoch 100, Loss: 3.969155353652236
Epoch 200, Loss: 3.9687816044840267
Epoch 300, Loss: 3.9684078741225504
Epoch 400, Loss: 3.968034162650691
Epoch 500, Loss: 3.9676604701513063
Epoch 600, Loss: 3.967286796707237
Epoch 700, Loss: 3.966913142401296
Epoch 800, Loss: 3.9665395073162766
Epoch 900, Loss: 3.966165891534948
Root Mean Square Error (RMSE): 1.9711


## 5.6 - Regression 5 layers

In [ ]:
nn_regression_5Layer = multi_purpose_NN(
    alpha=0.01,
    activation_func='sigmoid',
    error_func='MSE',
    n_neurons=36,
    n_features=8,
    n_instances=X_train_reg.shape[0],
    n_layers=5
)

# Train the regression model
epochs = 1000
nn_regression_5Layer.train(X_train_reg, y_train_reg, epochs)

# Make predictions using the trained model
y_pred_reg = nn_regression_5Layer.predict(X_test_reg)

# Calculate RMSE using the method in your class
rmse = nn_regression_5Layer.calculate_rmse(y_test_reg, y_pred_reg)

# Display RMSE
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")

Epoch 0, Loss: 3.9400443333349826
Epoch 100, Loss: 3.939631410111724
Epoch 200, Loss: 3.9392185183926847
Epoch 300, Loss: 3.938805658288919
Epoch 400, Loss: 3.938392829911435
Epoch 500, Loss: 3.937980033371193
Epoch 600, Loss: 3.9375672687791075
Epoch 700, Loss: 3.9371545362460476
Epoch 800, Loss: 3.9367418358828328
Epoch 900, Loss: 3.9363291678002392
Root Mean Square Error (RMSE): 1.9636
